##Célula 1
Nesta etapa centralizamos todas as bibliotecas usadas ao longo do projeto e carregamos o dataset.

Seguir esse caminho garante:

*   melhor organização do notebook
*   facilidade de rastreamento de dependências
*   transparência sobre quais ferramentas são necessárias para execução

Também realizamos uma primeira inspeção do dataframe (head, info, describe) para:


*   entender o formato dos dados
*   confirmar tipos de variáveis
*   verificar possíveis problemas como ausências, tipos inadequados ou dispersão inesperada

Essa visão inicial fundamenta todas as decisões posteriores de limpeza e modelagem.

In [3]:
# CÉLULA 1 - IMPORTS E CARREGAMENTO DO CSV

import pandas as pd
import gradio as gr
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.optimize import minimize


# Carregar o dataset
df = pd.read_csv('banco_solos_integrado_1000_linhas_v2.csv')

print("Primeiras 5 linhas:")
print(df.head())

print("\nInformações do dataframe:")
df.info()

print("\nEstatísticas descritivas:")
print(df.describe())


Primeiras 5 linhas:
   id  cultura  tipo_solo    ph  umidade_pct  irrigacao_horas       n_mgkg  \
0   1  laranja  misturado  4.98         29.4              4.1  1156.398451   
1   2     cana  misturado  6.31         20.5              2.0  1026.618987   
2   3  laranja    massape  5.45         27.7              0.4  1001.713175   
3   4  laranja        NaN  7.10         12.0              5.8  1226.116394   
4   5     cana    arenoso  7.48         31.0              4.5   691.909007   

      p_mgkg      k_mgkg  producao_ton_ha  
0  41.753934         NaN        32.656532  
1  49.053094  210.538752        84.332550  
2  25.122190  122.188102        27.831272  
3  53.160568  196.893933        27.564167  
4  73.674936  125.874616       104.762201  

Informações do dataframe:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               

##Célula 2
Antes de modelar, asseguramos que não existam valores vazios na base, pois:

*   algoritmos de machine learning não aceitam NaN
*   dados faltantes podem distorcer os resultados

Optamos por:

*   moda para variáveis categóricas, pois resgata o comportamento mais frequente
*   mediana para variáveis numéricas, pois é mais robusta em relação a outliers que a média

Com isso, garantimos consistência do dataset sem criar valores artificiais que possam afetar o comportamento dos modelos.

In [4]:
# CÉLULA 2 - TRATAMENTO DE VALORES AUSENTES

categorical_cols_with_na = ['cultura', 'tipo_solo']
numerical_cols_with_na = ['ph', 'umidade_pct', 'irrigacao_horas',
                          'n_mgkg', 'p_mgkg', 'k_mgkg', 'producao_ton_ha']

# Preencher categóricas com a moda
for col in categorical_cols_with_na:
    if df[col].isnull().any():
        df[col] = df[col].fillna(df[col].mode()[0])

# Preencher numéricas com a mediana
for col in numerical_cols_with_na:
    if df[col].isnull().any():
        df[col] = df[col].fillna(df[col].median())

print("\nValores ausentes após preenchimento:")
print(df.isnull().sum())



Valores ausentes após preenchimento:
id                 0
cultura            0
tipo_solo          0
ph                 0
umidade_pct        0
irrigacao_horas    0
n_mgkg             0
p_mgkg             0
k_mgkg             0
producao_ton_ha    0
dtype: int64


##Célula 3
Convertendo cultura e tipo de solo em variáveis binárias via get_dummies, possibilitamos:


* que modelos numéricos compreendam variáveis categóricas
* preservação das relações de cada classe com o resultado

Também optamos por drop_first=True para evitar multicolinearidade, tornando o modelo mais estável e interpretável.

In [5]:
# CÉLULA 3 - ONE-HOT ENCODING

df = pd.get_dummies(df, columns=['cultura', 'tipo_solo'], drop_first=True, dtype=int)

print("Primeiras 5 linhas após one-hot:")
print(df.head())


Primeiras 5 linhas após one-hot:
   id    ph  umidade_pct  irrigacao_horas       n_mgkg     p_mgkg      k_mgkg  \
0   1  4.98         29.4              4.1  1156.398451  41.753934  151.024496   
1   2  6.31         20.5              2.0  1026.618987  49.053094  210.538752   
2   3  5.45         27.7              0.4  1001.713175  25.122190  122.188102   
3   4  7.10         12.0              5.8  1226.116394  53.160568  196.893933   
4   5  7.48         31.0              4.5   691.909007  73.674936  125.874616   

   producao_ton_ha  cultura_laranja  cultura_soja  tipo_solo_argiloso  \
0        32.656532                1             0                   0   
1        84.332550                0             0                   0   
2        27.831272                1             0                   0   
3        27.564167                1             0                   0   
4       104.762201                0             0                   0   

   tipo_solo_latossolo  tipo_solo_massape

##Célula 4
Aqui definimos X (entradas) e y (resultado), removendo:

*   a coluna alvo
*   identificadores como id, pois não contribuem para o aprendizado

Fazemos o train_test_split para:

*   simular como o modelo performaria com dados nunca vistos
*   evitar overfitting
*   manter rigor no processo de avaliação

Essa separação garante que o desempenho medido seja mais realista e confiável.

In [6]:
# CÉLULA 4 - FEATURE SELECTION E TRAIN/TEST SPLIT

target_column = 'producao_ton_ha'

X = df.drop(columns=[col for col in ['id', target_column] if col in df.columns])
y = df[target_column]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"X_train: {X_train.shape}, X_test: {X_test.shape}")
print(f"y_train: {y_train.shape}, y_test: {y_test.shape}")


X_train: (800, 12), X_test: (200, 12)
y_train: (800,), y_test: (200,)


##Célula 5
Testamos quatro algoritmos supervisionados: Linear Regression, Random Forest, Gradient Boosting e XGBoost.

Esse conjunto misturam modelos lineares simples e ensembles não lineares de grande capacidade.

Realizamos as análises de desempenho MAE,MSE, RMSE e R².

Comparar múltiplos algoritmos aumenta a confiabilidade do resultado, pois permite escolher aquele que melhor se adapta ao padrão real dos dados.

In [7]:
# CÉLULA 5 - TREINAMENTO DE 4 MODELOS

models_to_train = {
    'LinearRegression': LinearRegression(),
    'RandomForestRegressor': RandomForestRegressor(random_state=42),
    'GradientBoostingRegressor': GradientBoostingRegressor(random_state=42),
    'XGBRegressor': xgb.XGBRegressor(random_state=42)
}

performance = {}
trained_models = {}

for name, model in models_to_train.items():

    model.fit(X_train, y_train)
    trained_models[name] = model

    pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, pred)
    mse = mean_squared_error(y_test, pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, pred)

    performance[name] = {
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'R2': r2
    }

performance


{'LinearRegression': {'MAE': 6.285930240294183,
  'MSE': 110.2952540635683,
  'RMSE': 10.502154734318491,
  'R2': 0.9132433518335449},
 'RandomForestRegressor': {'MAE': 6.54514821339872,
  'MSE': 133.66659854614255,
  'RMSE': 11.56142718465772,
  'R2': 0.8948597910206463},
 'GradientBoostingRegressor': {'MAE': 6.626743329726626,
  'MSE': 122.68158568622182,
  'RMSE': 11.076171977999522,
  'R2': 0.9035004429134543},
 'XGBRegressor': {'MAE': 7.674577445477233,
  'MSE': 156.02211099080301,
  'RMSE': 12.490881113468458,
  'R2': 0.8772752689647435}}

##Célula 6
Selecionamos o melhor modelo com base no maior valor de R², que indica quanta variabilidade do resultado o modelo consegue explicar mesmo quando aplicado sobre dados novos.

Armazenamos o melhor modelo para uso nas próximas etapas.

Esse processo reforça:
*   rastreabilidade
*   reprodutibilidade
*   uso consistente do modelo mais eficiente do pipeline

In [8]:
# CÉLULA 6 - IDENTIFICAR O MELHOR MODELO

best_model_name = max(performance, key=lambda m: performance[m]['R2'])
best_model = trained_models[best_model_name]

print(f"\nMelhor modelo: {best_model_name}")
print(performance[best_model_name])



Melhor modelo: LinearRegression
{'MAE': 6.285930240294183, 'MSE': 110.2952540635683, 'RMSE': 10.502154734318491, 'R2': 0.9132433518335449}


#Célula 7
Criamos uma interface interativa que permite ao usuário escolher cultura e tipo de solo, executa a otimização personalizada com base nesses fatores e retorna:

*   PH ideal
*   Umidade ideal
*   Irrigação ideal
*   N, P e K ideais
*   produção máxima estimada

Com isso, o notebook deixa de ser apenas uma análise técnica e se transforma em:


*   protótipo de sistema
*   ferramenta de tomada de decisão
*   simulador para agrônomos, pesquisadores e produtores

Essa interface demonstra o valor aplicado do modelo ao mundo real.

In [10]:
# CÉLULA 7 - INTERFACE GRADIO

import gradio as gr
from scipy.optimize import minimize

# ---------------------------
# Variáveis de otimização (somente as numéricas)
# ---------------------------
optimization_features = [
    'ph',
    'umidade_pct',
    'irrigacao_horas',
    'n_mgkg',
    'p_mgkg',
    'k_mgkg'
]

# ---------------------------
# Limites (bounds) obtidos do próprio dataset
# ---------------------------
feature_bounds = {
    col: {'min': df[col].min(), 'max': df[col].max()}
    for col in optimization_features
}

# ---------------------------
# Ordem das colunas usadas pelo modelo
# ---------------------------
feature_order = list(X.columns)

# ---------------------------
# Listas para os dropdowns
# (cultura e tipo_solo antes do one-hot encoding)
# ---------------------------
cultura_options = sorted(df.filter(like='cultura_').columns)
cultura_options = [c.split('cultura_')[1] for c in cultura_options]
cultura_options.insert(0, 'milho')      # garantindo base

tipo_solo_options = sorted(df.filter(like='tipo_solo_').columns)
tipo_solo_options = [t.split('tipo_solo_')[1] for t in tipo_solo_options]
tipo_solo_options.insert(0, 'arenoso')  # garantindo base


# ---------------------------
# Função de otimização
# ---------------------------
def otimizar_parametros(cultura, tipo_solo):

    # Valores iniciais (ponto médio dos limites)
    init_guess = [
        (feature_bounds[f]['min'] + feature_bounds[f]['max']) / 2
        for f in optimization_features
    ]

    # Construção base dos valores fixos
    fixed_values = {col: 0 for col in feature_order if col not in optimization_features}

    # Marca a cultura selecionada
    if cultura != 'milho':
        col_c = f'cultura_{cultura}'
        if col_c in fixed_values:
            fixed_values[col_c] = 1

    # Marca o tipo de solo selecionado
    if tipo_solo != 'arenoso':
        col_s = f'tipo_solo_{tipo_solo}'
        if col_s in fixed_values:
            fixed_values[col_s] = 1

    # Função a ser maximizada (negativa pois minimizamos)
    def objective(values):

        # monta entrada para o modelo
        inp = {f: v for f, v in zip(optimization_features, values)}
        inp.update(fixed_values)

        df_inp = pd.DataFrame([inp], columns=feature_order)

        # retorna negativo para maximizar
        return -best_model.predict(df_inp)[0]

    # limites numéricos
    bounds = [
        (feature_bounds[f]['min'], feature_bounds[f]['max'])
        for f in optimization_features
    ]

    # executa otimização
    result = minimize(
        objective,
        init_guess,
        method='L-BFGS-B',
        bounds=bounds
    )

    # prepara resultado
    output = {f: v for f, v in zip(optimization_features, result.x)}
    output['Produção Máxima (ton/ha)'] = -result.fun

    return (
        round(float(output['ph']), 2),
        round(float(output['umidade_pct']), 2),
        round(float(output['irrigacao_horas']), 2),
        round(float(output['n_mgkg']), 2),
        round(float(output['p_mgkg']), 2),
        round(float(output['k_mgkg']), 2),
        round(float(output['Produção Máxima (ton/ha)']), 2)
    )


# ---------------------------
# Interface Gradio
# ---------------------------
iface = gr.Interface(
    fn=otimizar_parametros,
    inputs=[
        gr.Dropdown(cultura_options, label="Cultura"),
        gr.Dropdown(tipo_solo_options, label="Tipo de Solo")
    ],
    outputs=[
        gr.Textbox(label="PH ideal"),
        gr.Textbox(label="Umidade ideal (%)"),
        gr.Textbox(label="Irrigação ideal (horas)"),
        gr.Textbox(label="Nitrogênio ideal (mg/kg)"),
        gr.Textbox(label="Fósforo ideal (mg/kg)"),
        gr.Textbox(label="Potássio ideal (mg/kg)"),
        gr.Textbox(label="Produção Máxima Prevista (ton/ha)")
    ],
    title="Otimização de Parâmetros Agrícolas",
    description="Simule os parâmetros ideais de manejo agrícola com base no modelo que apresentou melhor desempenho."
)

iface.launch(debug=True)


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
